# Original reference material

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import sys

import torch
from torch import nn
from torch import optim

print(sys.version)
print(torch.__version__)
print(torch.version.cuda)

In [ ]:
class policy_estimator():
    def __init__(self, env):
        self.n_inputs = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        
        # Define network
        self.network = nn.Sequential(
            nn.Linear(self.n_inputs, 16), 
            nn.ReLU(), 
#             nn.Linear(32, 16),
#             nn.ReLU(),        
            nn.Linear(16, self.n_outputs),
            nn.Softmax(dim=-1))
    
    def predict(self, state):
        action_probs = self.network(torch.FloatTensor(state))
        return action_probs

In [ ]:
# env = gym.make('CartPole-v0')
# env = gym.make("Acrobot-v1")
env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000
# add in 1000 steps if this doesn't work!

s = env.reset()
pe = policy_estimator(env)
print(pe.predict(s))
print(pe.network(torch.FloatTensor(s)))

In [ ]:
def discount_rewards(rewards, gamma=0.99):
    r = np.array([gamma**i * rewards[i] 
                  for i in range(len(rewards))])
    # Reverse the array direction for cumsum and then
    # revert back to the original order
    r = r[::-1].cumsum()[::-1]
    return r - r.mean()

In [ ]:

def reinforce(env, policy_estimator, num_episodes=2000,
              batch_size=10, gamma=0.99, lr = 0.01):

    # Set up lists to hold results
    total_rewards = []
    batch_rewards = []
    batch_actions = []
    batch_states = []
    batch_counter = 1
    
    # Define optimizer
    optimizer = optim.Adam(policy_estimator.network.parameters(), 
                           lr=lr)
    
    action_space = np.arange(env.action_space.n)
    for ep in range(num_episodes):
        s_0 = env.reset().unsqueeze(0)
        states = []
        rewards = []
        actions = []
        complete = False
        while complete == False:
            # Get actions and convert to numpy array
            action_probs = policy_estimator.predict(s_0).detach().numpy()
            action = np.random.choice(action_space, p=action_probs)
            s_1, r, complete, _ = env.step(action)
            
            states.append(s_0)
            rewards.append(r)
            actions.append(action)
            s_0 = s_1
            
            # If complete, batch data
            if complete:
                batch_rewards.extend(discount_rewards(rewards, gamma))
                batch_states.extend(states)
                batch_actions.extend(actions)
                batch_counter += 1
                total_rewards.append(sum(rewards))
                
                # If batch is complete, update network
                if batch_counter == batch_size:
                    optimizer.zero_grad()
                    state_tensor = torch.FloatTensor(batch_states)
                    reward_tensor = torch.FloatTensor(batch_rewards)
                    # Actions are used as indices, must be LongTensor
                    action_tensor = torch.LongTensor(batch_actions)
                    
                    #Moving average baseline
                    
      
                    # Basically create a tensor size [1, len(reward_tensor)] Below doesn't actually work
                    WINDOW = 20
                    baseline = np.mean(total_rewards[-WINDOW:])
            
                                  
                    # Calculate loss
                    reward_with_baseline = reward_tensor - baseline
                                 
                    logprob = torch.log(
                        policy_estimator.predict(state_tensor))
                    
                    selected_logprobs = reward_with_baseline * \
                        logprob[np.arange(len(action_tensor)), action_tensor]
                    
                    
                    # regular reward (should be higher variance. But how do I calculate this?)
#                     selected_logprobs = reward_tensor * \
#                         logprob[np.arange(len(action_tensor)), action_tensor]
                    loss = -selected_logprobs.mean()
    
    
#                     policy_estimator.zero_grad()     # zeroes the gradient buffers of all parameters. Prevents the gradients from adding up
                    
                    # Calculate gradients
                    loss.backward()
                    # Apply gradients
                    optimizer.step()
                    
                    batch_rewards = []
                    batch_actions = []
                    batch_states = []
                    batch_counter = 1
                    
                # Print running average
                print("\rEp: {} Average of last 10: {:.2f}".format(
                    ep + 1, np.mean(total_rewards[-10:])), end="")
                
    return total_rewards

In [ ]:
import time
# env = gym.make('CartPole-v0')
# env = gym.make("Acrobot-v1")
# env = gym.make("MountainCar-v0")
# env._max_episode_steps = 1000
# add in 1000 steps if this doesn't work!
# seed = env.seed()
# pe = policy_estimator(env)


fig, axs = plt.subplots(5,1, figsize = (12,40))
for run in range(5):
    # Create a new environment instance (seed is different at each run)
    env = gym.make("MountainCar-v0")
    env._max_episode_steps = 1000
    seed = env.seed()
    pe = policy_estimator(env)
    
    
    start = time.time()
    rewards = reinforce(env, pe)
    
    end = time.time()
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    window = 10
    smoothed_rewards = [np.mean(rewards[i-window:i+1]) if i > window 
                        else np.mean(rewards[:i+1]) for i in range(len(rewards))]
    
    axs[run].plot(rewards)
    axs[run].plot(smoothed_rewards)
    axs[run].set_ylabel('Total Rewards')
    axs[run].set_xlabel('Episodes')
    
#     plt.figure(figsize=(12,8))
#     plt.plot(rewards)
#     plt.plot(smoothed_rewards)
#     plt.ylabel('Total Rewards')
#     plt.xlabel('Episodes')
plt.show()

In [ ]:
fig.savefig("policygradient_mountainCar-1.png")

# My best implementation (with this guide above as reference)

In [2]:
import numpy as np
import torch.nn.functional as F
from torch import nn
from torch import optim
import torch
print(torch.__version__)
import gym
from evaluation import evaluate
# For sanity, have set the seed on all possible variables





# Input differentiable policy parameterization & Step size
STEP_SIZE = 0.01


class PolicyEstimator(nn.Module):
    def __init__(self, env):
        super(PolicyEstimator,self).__init__()
        
        self.n_input = env.observation_space.shape[0]
        self.n_output = env.action_space.n
        self.fc1 = nn.Linear(self.n_input, 16)
        self.fc2 = nn.Linear(16, self.n_output)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim = -1)
        return x

    
# Test of output
env = gym.make("MountainCar-v0")

state = torch.tensor(env.reset()).float()
print(state)
pe = PolicyEstimator(env)
print(pe(state).sum()) # Softmax that should add up to 1

1.1.0
tensor([-0.5814,  0.0000])
tensor(1., grad_fn=<SumBackward0>)


In [34]:
# Initialize Policy parameter theta (e.g. to 0 vector) -- currently I am going with assumption that they are already initialized
# help(pe.zero_grad) # This is also something that I didn't do. This could be important (to prevent building up of gradients)
print(pe.parameters())

<generator object Module.parameters at 0x000001D3EFC64048>


In [81]:
type(env.reset())

numpy.ndarray

In [3]:
#Pick an action

# MAKE SURE that you have things seeded. That likely makes things easier. Randomize the seed that you choose I guess. Is there another way to do it?
np.random.seed(0)
torch.manual_seed(0)
env.seed(0)

def action_choice(policy_estimator, state):
    # May cause conflicts with the Maze environment
    print(state)
    action_probs = policy_estimator(state).detach().numpy()
    action = np.random.choice((policy_estimator.n_output), p = action_probs)
    idx = action_probs[action]
    print(action_probs)
    return action, idx


state = torch.from_numpy(env.reset()).float()
print(action_choice(pe, state))

tensor([-0.5891,  0.0000])
[0.3804588  0.314339   0.30520222]
(1, 0.314339)


In [4]:
#Pick an action
def action_choice(policy_estimator, state):
    # May cause conflicts with the Maze environment
    action_probs = policy_estimator(state).detach().numpy()
    action = np.random.choice((policy_estimator.n_output), p = action_probs)
    idx = action_probs[action]
    return action, idx


# Generate an episode (loop forever [actually until episode is done, because it's continuous])

done = False
env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000
seed = env.seed(0)
action_prob_val = []
action_list = []
state_list = []
s_0 = torch.from_numpy(env.reset()).float() # Initial state
cumulative_reward = 0
reward_list = []
while not done:
    # Pick an action, and step through environment
    action, prob = action_choice(pe, s_0)
    
    # Append this before. Then you'll get the same size
    state_list.append(s_0)
    
    s_1, reward, done, _ = env.step(action)
    cumulative_reward += reward
    action_prob_val.append(prob)
    reward_list.append(reward)
    action_list.append(action)
    s_0 = torch.from_numpy(s_1).float() # Make state the next one
    if done:
        print(cumulative_reward) # for a single episode

        
print(len(reward_list))
print(len(action_list))
print(len(state_list))
print(len(action_prob_val))



-1000.0
1000
1000
1000
1000


# Integration Testing portion

In [5]:
# Integration testing. Goal is to get all the way back to the parameters of NN. The reason the other person re-did all the tensors was because of backprop in Pytorch

# Change of plans, will start from the beginning

reward_tensor = torch.tensor(reward_list).float()
action_tensor = torch.tensor(action_list).long()
state_tensor = torch.stack(state_list)

print(reward_tensor.shape)
print(action_tensor.shape)
print(state_tensor.shape)

# Goal is to make sure that when estimating, you are simulating the same conditions
# I don't think that setting the seed is the right way to do it. I prob should confirm whether I get the same value or not.


# Does this need to be seeded?
action_output = torch.log(pe(state_tensor))

# This is just doing an indexing! It picks the action that was chosen in the run. There is no operation here that pytorch has to track
picked_logprob = action_output[np.arange(len(action_tensor)), action_tensor]

print(picked_logprob[:10])
print(action_tensor[:10])


torch.Size([1000])
torch.Size([1000])
torch.Size([1000, 2])
tensor([-1.1868, -1.1573, -1.1572, -1.1570, -1.1567, -1.1564, -1.1858, -1.1854,
        -1.1552, -1.1845], grad_fn=<SliceBackward>)
tensor([2, 1, 1, 1, 1, 1, 2, 2, 1, 2])


In [8]:
# Get the return values as well

return_tensor = discount_rewards(reward_tensor)
# print(return_tensor[-10:])

loss = torch.sum(-return_tensor * picked_logprob)
print(loss)

loss.backward()

tensor(-98241.0859, grad_fn=<SumBackward0>)


In [7]:
def discount_rewards(reward_array, DISCOUNT = 0.99):
    '''
    This will take the reward, and recursively update them, in reverse.
    
    input: reward_array[list]: list of rewards that are generated per episode
    
    '''
    
#     reward_array = torch.FloatTensor(reward_array)
    G = torch.zeros((len(reward_array),)).float()
    cumulative = 0
    for i in reversed(range(len(reward_array)-1)):
        G[i] = reward_array[i+1] + DISCOUNT * G[i+1]
    return G

# Create the discounted goal at each step

G = discount_rewards(reward_list)
print(G)



tensor([-99.9957, -99.9957, -99.9956, -99.9956, -99.9956, -99.9955, -99.9955,
        -99.9954, -99.9954, -99.9953, -99.9953, -99.9952, -99.9952, -99.9951,
        -99.9951, -99.9950, -99.9950, -99.9949, -99.9949, -99.9948, -99.9948,
        -99.9947, -99.9947, -99.9946, -99.9946, -99.9945, -99.9944, -99.9944,
        -99.9943, -99.9943, -99.9942, -99.9941, -99.9941, -99.9940, -99.9940,
        -99.9939, -99.9938, -99.9938, -99.9937, -99.9937, -99.9936, -99.9935,
        -99.9935, -99.9934, -99.9933, -99.9932, -99.9932, -99.9931, -99.9930,
        -99.9930, -99.9929, -99.9928, -99.9928, -99.9927, -99.9926, -99.9925,
        -99.9924, -99.9924, -99.9923, -99.9922, -99.9921, -99.9921, -99.9920,
        -99.9919, -99.9918, -99.9917, -99.9916, -99.9916, -99.9915, -99.9914,
        -99.9913, -99.9912, -99.9911, -99.9910, -99.9909, -99.9908, -99.9907,
        -99.9907, -99.9906, -99.9905, -99.9904, -99.9903, -99.9902, -99.9901,
        -99.9900, -99.9899, -99.9898, -99.9897, -99.9895, -99.98

In [ ]:
# Compute loss function. The policy part - is the output a probability or the action it represents
# action_prob_val = torch.tensor(action_prob_val).float()
# log_probs = -G * action_prob_val



log_probs = -torch.sum(G) # This will return nan for negative number! wtf. B/c I forgot what a log graph looks like


In [3]:
from evaluation import evaluate
%matplotlib inline
import matplotlib.pyplot as plt
import importlib
# importlib.reload(evaluation)



def generate_episode(env, policy_estimator):
    '''
    Useful for reinforcementl earning tasks with Monte Carlo tasks. This function is specifcally for policy gradient methods
    
    Input: env: Environment
    policy_estimator: function approximator. Specifically Neural network
    '''
    # Initial state
    
    
    # Record actions, states, and rewards for each episode
    states = []
    actions = []
    rewards = []
    done = False



    # If it isn't the x episode, then continue the evaluation

    s_0 = torch.tensor(env.reset()).float()

    while done is False:

        action = action_choice(policy_estimator, s_0)
        s_1, reward, done, _ = env.step(action)


        states.append(s_1)
        actions.append(action)
        rewards.append(reward)

        s_1 = torch.from_numpy(s_1).float()
        s_0 = s_1 # I missed this, and this is really important!!!!!
        
        if done:
            return states, actions, rewards
    
def action_choice(policy_estimator, state):
    # May cause conflicts with the Maze environment
    state = state.unsqueeze(0)[0]
    action_probs = policy_estimator(state).detach().numpy()
    action = np.random.choice((policy_estimator.n_output), p = action_probs)
    return action
    
def discount_rewards(reward_array, DISCOUNT = 0.99):
    '''
    This will take the reward, and recursively update them, in reverse.
    
    '''
    
#     reward_array = torch.FloatTensor(reward_array)
    G = torch.zeros((len(reward_array),)).float()
    cumulative = 0
    for i in reversed(range(len(reward_array)-1)):
        G[i] = reward_array[i+1] + DISCOUNT * G[i+1]
    return G

def reinforce(env, policy_estimator, DISCOUNT = 0.99, EPISODES = 2000, lr = 0.01, EVALUATION_STEP = 400):
    # Define optimizer
    optimizer = optim.Adam(policy_estimator.parameters(), lr = lr)

    cumulative_reward = []
    running_reward = 0
    
    steps = []
    eval_reward = []
        
    
    # Update over X number of episodes (default 2000)
    for ep in range(EPISODES):
        
        # Evaluate every Y episodes (default 400). Add reward step each iteration
        if ep % EVALUATION_STEP == 0:
            st, er = evaluate(env, Q_table = None, step_bound = 100, num_itr = 10, Gym = True, policy_estimator = policy_estimator)
            steps.append(st)
            eval_reward.append(er)

        # Generate episode, save all states inside the episode
        states, actions, rewards = generate_episode(env, policy_estimator)
        
        # Record running reward
        running_reward += sum(rewards)
        cumulative_reward.append(running_reward)
        
        
        # Create discounted reward array for each iteration of episode
        G = discount_rewards(rewards, DISCOUNT)

        
        
        # Convert arrays to tensor form
        
        
        action_tensor = torch.LongTensor(actions)
        states_tensor = torch.tensor(states).float()
        reward_tensor = torch.tensor(G).float()

        

# #############################################
#         Baseline##########
#         WINDOW = 20
#         baseline = np.mean(reward_tensor[-WINDOW:].detach().numpy())
#         reward_with_baseline = reward_tensor - baseline
#           selected_logprobs = reward_with_baseline * logprob[np.arange(len(action_tensor)), action_tensor]
# #############################################      
        
    # Find the loss function
        logprob = torch.log(policy_estimator(states_tensor))
        selected_logprobs = reward_tensor * logprob[np.arange(len(action_tensor)), action_tensor]
        loss = -selected_logprobs.sum()
        
        print("\r Ep {}/{} running reward: {}, loss {}".format(ep, EPISODES, running_reward, loss), end = "")
        
        #Backprop & update step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Clear values for new episode
        states = []
        actions = []
        rewards = []

    return cumulative_reward, steps, eval_reward
                
                
                
        
        
        
        
    
    

In [4]:
# Cumulative reward

env = gym.make("CartPole-v0")
net = PolicyEstimator(env)
cum_reward = reinforce(env, net)
print(cum_reward[-10:])

ValueError: object too deep for desired array

In [ ]:
# 5 different seeds


fig, axs = plt.subplots(5,1, figsize = (12,40))
for run in range(5):
    # Create a new environment instance (seed is different at each run)
    env = gym.make("MountainCar-v0")
    env._max_episode_steps = 1000
    seed = env.seed()
    pe = policy_estimator(env)
    
    
    start = time.time()
    rewards = reinforce(env, pe)
    
    end = time.time()
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    window = 10
    smoothed_rewards = [np.mean(rewards[i-window:i+1]) if i > window 
                        else np.mean(rewards[:i+1]) for i in range(len(rewards))]
    
    axs[run].plot(rewards)
    axs[run].plot(smoothed_rewards)
    axs[run].set_ylabel('Total Rewards')
    axs[run].set_xlabel('Episodes')



    
    

In [ ]:
# 
fig, axs = plt.subplots(1,1,figsize= (15,11))
seeds = []

import time
import matplotlib.pyplot as plt


for run in range(5):
    env = gym.make("Acrobot-v1")
#     env._max_episode_steps = 1000
    seed = env.seed()
    seeds.append(seed)
    pe = PolicyEstimator(env)
    
    
    start = time.time()
    cum_reward = reinforce(env, pe)
    
    end = time.time()
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    axs.plot(cum_reward)
    axs.set_ylabel("Cumulative reward")
    axs.set_xlabel("Episodes")
    axs.legend(seeds)
plt.show()

# Attempt with class evaluation script

In [ ]:
import time
import matplotlib.pyplot as plt

RUNS = 2 # number of different seeds you are trying out - checking by eye for variance


fig, axs = plt.subplots(RUNS,1,figsize= (15 * RUNS,11))
seeds = []




for run in range(RUNS):
    env = gym.make("Acrobot-v1")
#     env._max_episode_steps = 1000
    seed = env.seed()
    seeds.append(seed)
    pe = PolicyEstimator(env)
    
    
    start = time.time()
    cum_reward, steps, eval_reward = reinforce(env, pe, EPISODES = 2000)
    
    end = time.time()
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    axs[0].plot(steps)
    axs[0].set_ylabel("Steps taken")
    axs[0].set_xlabel("Episodes")
    axs[0].legend(seeds)
    
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    axs[1].plot(eval_reward)
    axs[1].set_ylabel("Cumulative reward")
    axs[1].set_xlabel("Episodes")
    axs[1].legend(seeds)
plt.show()

In [ ]:
print(steps)
print(eval_reward)

In [ ]:
%whos




In [5]:
from maze import *
import gym
import time
import sys
import numpy as np
import argparse
import evaluation
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

class MDP(Maze):
    '''
    Inherits all the attributes from the maze
    '''
    def __init__(self):
        super().__init__()
        # self.LEARNING_RATE = 0.1

        
        
        
class PolicyEstimator(nn.Module):
    '''
    Feedforward network definition for maze environment
    
    '''
    def __init__(self, env):
        super(PolicyEstimator,self).__init__()

        self.n_input = 1
        self.n_output = env.anum


        self.fc1 = nn.Linear(self.n_input, 16)
        self.fc2 = nn.Linear(16, self.n_output)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim = -1)
        return x        
    
    

    

env = MDP()

state = torch.tensor(env.reset()).float().unsqueeze(0) # unsqueeze adds a fake batch dimension
print(state)
net = PolicyEstimator(env)
output = net(state)
print(output)

tensor([0.])
tensor([0.2792, 0.3435, 0.1829, 0.1944], grad_fn=<SoftmaxBackward>)


In [10]:
env = MDP()
print(type(env.reset()))
env2 = gym.make("CartPole-v0")
print(type(env2.reset()))

<class 'int'>
<class 'numpy.ndarray'>


In [ ]:

evaluate(env, policy_estimator= net)

In [ ]:
RUNS = 2
env = 
for run in range(RUNS):
#     env = gym.make("Acrobot-v1")
#     env._max_episode_steps = 1000
#     seed = env.seed()
#     seeds.append(seed)
    pe = PolicyEstimator(env)
    
    
    start = time.time()
    cum_reward, steps, eval_reward = reinforce(env, pe, EPISODES= 2000)
    
    end = time.time()
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    axs[0].plot(steps)
    axs[0].set_ylabel("Steps taken")
    axs[0].set_xlabel("Episodes")
#     axs[0].legend(seeds)
    
    print(f'time of run {run} for seed {seed} in minutes: {(end-start) / 60.}')
    axs[1].plot(eval_reward)
    axs[1].set_ylabel("Cumulative reward")
    axs[1].set_xlabel("Episodes")
#     axs[1].legend(seeds)
plt.show()